# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
import re

def extract_floats(s):
    """
    Extracts all floats from a string and returns them as a list of strings.

    Parameters:
    - s (str): The string to extract floats from.

    Returns:
    - list: A list of strings, each representing a float found in the input string.
    """
    float_pattern = r"[-+]?\d*\.\d+|\d+"
    return [str(float(num)) for num in re.findall(float_pattern, s)]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data/rce_cohort_2"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2_both_rewarded"

In [10]:
COLS_TO_KEEP = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
'original_file', 'filename', 'session_path', 'all_subjects',
       'current_subject', 'event_timestamps', 'video_name',
       'video_timestamps', 'event_frames', 'first_item_data']

In [11]:
RAW_COLS_TO_KEEP = ['session_dir',
 'recording',
 'original_file',
 'session_path',
 'current_subject',
 'first_item_data',
 'first_timestamp',
 'all_subjects']

In [12]:
STATE_COLS_TO_KEEP = ['session_dir',
 'metadata_file',
 'event_timestamps',
 'video_name',
 'video_timestamps',
 'event_frames',]

In [13]:
same_columns = ['session_dir', 'video_name']
different_columns = ['metadata_file', 'event_frames', 'event_timestamps']

In [14]:
# TODO: Find way not to hard code this
# ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/standard/2023_06_*/*.rec")
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/rce_cohort_2/both_rewarded/*.rec")



In [15]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/rce_cohort_2/both_rewarded/20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/both_rewarded/20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4.rec',
 '/scratch/back_up/reward_competition_extention/data/rce_cohort_2/both_rewarded/20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [16]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [17]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = utilities.helper.create_recursive_dict()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4


/nancy/user/riwata/projects/reward_comp_ext/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4
Skipping file 20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-1_t1b2L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-4_t3b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Current Session: 20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2
Skipping file 20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230622_110832_standard_comp_to_both_rewar

In [18]:
session_to_trodes_data

defaultdict(<function utilities.helper.create_recursive_dict()>,
            {'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4': defaultdict(dict,
                         {'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged': {'analog': {'timestamps': {'description': 'Analog IO timestamps',
                             'byte_order': 'little endian',
                             'original_file': '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                             'headstage_firmware': '2.2',


- Adding the video timestamps

In [19]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            if "video_timestamps" not in session_to_trodes_data[session_basename][session_basename]:
                session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4
Current Video Name: 20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.1.videoTimeStamps.cameraHWSync
Current Session: 20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4
Current Video Name: 20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4.1.videoTimeStamps.cameraHWSync
Current Session: 20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2
Current Video Name: 20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync


- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [20]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [21]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,fields,data,filename,decimation,direction,id,display_order,clock rate,session_path
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,analog,timestamps,Analog IO timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,6771359,<time uint32>,"[[6771359], [6771360], [6771361], [6771362], [...",20230625_112913_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,time,timestamps,Timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,6771359,<time uint32><systime int64>,"[[6771359, 1687707064530708500], [6771360, 168...",20230625_112913_standard_comp_to_both_rewarded...,20,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,coordinates,Pad locations in microns,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,6771359,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230625_112913_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,6771359,<time uint32>,"[[6771359], [6771360], [6771361], [6771362], [...",20230625_112913_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,6771359,<time uint32><state uint8>,"[[6771359, 0]]",20230625_112913_standard_comp_to_both_rewarded...,NaN,output,ECU_Dout1,2,NaN,/scratch/back_up/reward_competition_extention/...


In [22]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,fields,data,filename,decimation,direction,id,display_order,clock rate,session_path
70,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,4537337,<time uint32>,"[[4537337], [4537338], [4537339], [4537340], [...",20230622_110832_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
71,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,coordinates,Pad locations in microns,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,4537337,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230622_110832_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
72,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[6772743, 0, 0], [6774129, 0, 0], [6774129, 0...",20230625_112913_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...
73,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[8949944, 0, 0], [8950349, 0, 0], [8951330, 0...",20230624_105855_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...
74,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[4537335, 0, 0], [4538721, 0, 0], [4540107, 0...",20230622_110832_standard_comp_to_both_rewarded...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [23]:
trodes_metadata_df["data"].iloc[0]

array([( 6771359,), ( 6771360,), ( 6771361,), ..., (79195513,),
       (79195514,), (79195515,)], dtype=[('time', '<u4')])

In [24]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [25]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [26]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,decimation,direction,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,analog,timestamps,Analog IO timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",time,"[6771359, 6771360, 6771361, 6771362, 6771363, ..."
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,time,timestamps,Timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,20,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",systime,"[1687707064530708500, 1687707064530719500, 168..."
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,coordinates,Pad locations in microns,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",time,"[6771359, 6771360, 6771361, 6771362, 6771363, ..."
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,output,ECU_Dout1,2,NaN,/scratch/back_up/reward_competition_extention/...,time,[6771359],state,[0]


In [27]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,decimation,direction,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
70,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[4537337, 4537338, 4537339, 4537340, 4537341, ...",time,"[4537337, 4537338, 4537339, 4537340, 4537341, ..."
71,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,coordinates,Pad locations in microns,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
72,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
73,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8949944, 8950349, 8951330, 8952716, 8954102, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
74,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[4537335, 4538721, 4540107, 4541493, 4541493, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [28]:
trodes_metadata_df["recording"].unique()

array(['20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged',
       '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-4_t3b3L_box1_merged',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-1_t1b2L_box1_merged',
       '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-2_t3b3L_box1_merged',
       '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged',
       '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4',
       '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2'],
      dtype=object)

## Getting the subject information from the metadata

In [29]:
def split_by_multiple_delimiters(s, delimiters):
    """
    Splits a string by multiple delimiters.

    Parameters:
    - s (str): The string to split.
    - delimiters (list): A list of delimiters to split the string by.

    Returns:
    - list: A list of substrings.
    """
    return re.split('|'.join(map(re.escape, delimiters)), s)


In [30]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["session_dir"].apply(lambda x: x.split("subj")[-1].strip("_").replace("-", "."))#.split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted(extract_floats(x)))

In [31]:
trodes_metadata_df["session_dir"].iloc[0]

'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4'

In [32]:
trodes_metadata_df["all_subjects"].apply(lambda x: tuple(x)).unique()

array([('1.1', '1.4'), ('1.2', '1.4'), ('1.1', '1.2')], dtype=object)

In [33]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["recording"].apply(lambda x: x.split("subj")[-1].strip("_").replace("-", ".").replace("_", "."))#.split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["current_subject"] = trodes_metadata_df["current_subject"].apply(lambda x: str(extract_floats(x)[0]).strip())


In [34]:
trodes_metadata_df["current_subject"].unique()

array(['1.1', '1.4', '1.2'], dtype=object)

## Dropping all the rows with unneeded metadata

In [35]:
trodes_metadata_df["metadata_dir"].unique()

array(['analog', 'time', 'raw', 'DIO', 'video_timestamps'], dtype=object)

In [36]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [37]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [38]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [39]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [40]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [41]:
trodes_raw_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...","[1.1, 1.4]",1.1
9,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...","[1.1, 1.4]",1.4
14,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230624_105855_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[8948560, 8948561, 8948562, 8948563, 8948564, ...",time,"[8948560, 8948561, 8948562, 8948563, 8948564, ...","[1.2, 1.4]",1.4
19,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230624_105855_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[8948560, 8948561, 8948562, 8948563, 8948564, ...",time,"[8948560, 8948561, 8948562, 8948563, 8948564, ...","[1.2, 1.4]",1.1
20,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[4537337, 4537338, 4537339, 4537340, 4537341, ...",time,"[4537337, 4537338, 4537339, 4537340, 4537341, ...","[1.1, 1.2]",1.2


In [42]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [43]:
trodes_raw_df["recording"].iloc[0]

'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged'

In [44]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [45]:
recording_to_first_timestamp

{'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4': 6771359,
 '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4': 8948560,
 '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2': 4537337}

In [46]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [47]:
trodes_metadata_df["first_timestamp"]

0     6771359
1     6771359
2     6771359
3     6771359
4     6771359
5     6771359
6     6771359
7     6771359
8     6771359
9     6771359
10    8948560
11    8948560
12    8948560
13    8948560
14    8948560
15    8948560
16    8948560
17    8948560
18    8948560
19    8948560
20    4537337
21    4537337
22    4537337
23    4537337
24    4537337
25    4537337
26    4537337
27    4537337
28    4537337
29    4537337
30    6771359
31    8948560
32    4537337
Name: first_timestamp, dtype: int64

# Getting the event timestamps

In [48]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",time,"[6771359, 6771360, 6771361, 6771362, 6771363, ...","[1.1, 1.4]",1.1
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,[6771359],state,[1],"[1.1, 1.4]",1.1
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[6771359],state,[0],"[1.1, 1.4]",1.1
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1


In [49]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
28,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din4,9,NaN,/scratch/back_up/reward_competition_extention/...,time,[4537337],state,[0],"[1.1, 1.2]",1.1
29,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,raw,timestamps,Raw timestamps,little endian,20230622_110832_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[4537337, 4537338, 4537339, 4537340, 4537341, ...",time,"[4537337, 4537338, 4537339, 4537340, 4537341, ...","[1.1, 1.2]",1.1
30,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",1.1
31,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8949944, 8950349, 8951330, 8952716, 8954102, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",1.2
32,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[4537335, 4538721, 4540107, 4541493, 4541493, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.1


In [50]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()

# Filtering for digital IO channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO"])].copy()
# Filtering for tone and port entry related channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["id"].isin(["ECU_Din1", "ECU_Din2", "ECU_Din3"])].copy()


In [51]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din3,8,NaN,/scratch/back_up/reward_competition_extention/...,time,[6771359],state,[1],"[1.1, 1.4]",1.1
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1
6,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din2,6,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4
7,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,ECU_Din1,7,NaN,/scratch/back_up/reward_competition_extention/...,time,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4


In [52]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [53]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [54]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

## Updating the video timestamps

## Syncing up the video frame data

In [55]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)



In [56]:
# Filtering for the first video only
# This only applies to this pilot data where we are only looking the at competition data
# trodes_video_df = trodes_video_df[trodes_video_df["metadata_file"] == "1"].copy()

In [57]:
trodes_video_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,id,display_order,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.4]",1.1
1,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[8949944, 8950349, 8951330, 8952716, 8954102, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.2, 1.4]",1.2
2,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[4537335, 4538721, 4540107, 4541493, 4541493, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",1.1


In [58]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"]

In [59]:
# Removing the columns that are no longer needed
trodes_video_df = trodes_video_df[["filename", "video_timestamps", "session_dir"]].copy()

In [60]:
# Renaming the filename so that we can merge with other dataframes with the same column name
trodes_video_df = trodes_video_df.rename(columns={"filename": "video_name"})

In [61]:
trodes_video_df.head()

,video_name,video_timestamps,session_dir
0,20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",20230625_112913_standard_comp_to_both_rewarded...
1,20230624_105855_standard_comp_to_both_rewarded...,"[8949944, 8950349, 8951330, 8952716, 8954102, ...",20230624_105855_standard_comp_to_both_rewarded...
2,20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...",20230622_110832_standard_comp_to_both_rewarded...


- Adding each video as a row to each state row

In [62]:
trodes_state_df = pd.merge(trodes_state_df, trodes_video_df, on=["session_dir"], how="inner")

In [63]:
trodes_state_df.columns

Index(['session_dir', 'recording', 'metadata_dir', 'metadata_file',
       'description', 'byte_order', 'original_file', 'clockrate',
       'trodes_version', 'compile_date', 'compile_time', 'qt_version',
       'commit_tag', 'controller_firmware', 'headstage_firmware',
       'controller_serialnum', 'headstage_serialnum', 'autosettle', 'smartref',
       'gyro', 'accelerometer', 'magnetometer', 'time_offset',
       'system_time_at_creation', 'timestamp_at_creation', 'first_timestamp',
       'fields', 'data', 'filename', 'decimation', 'direction', 'id',
       'display_order', 'clock rate', 'session_path', 'first_dtype_name',
       'first_item_data', 'last_dtype_name', 'last_item_data', 'all_subjects',
       'current_subject', 'event_indexes', 'event_timestamps', 'video_name',
       'video_timestamps'],
      dtype='object')

## Finding the closest frame to each event

In [64]:
trodes_state_df["event_timestamps"].iloc[1]

array([[ 7874619,  8074622],
       [10274648, 10474653],
       [12574676, 12774678],
       [14374697, 14574699],
       [15474711, 15674713],
       [16474723, 16674725],
       [18374745, 18574748],
       [19874763, 20074765],
       [21974788, 22174790],
       [22974800, 23174805],
       [24574819, 24774821],
       [25874834, 26074837],
       [27174850, 27374852],
       [28174862, 28374864],
       [29574878, 29774881],
       [31474901, 31674903],
       [33074920, 33274922],
       [34474936, 34674939],
       [36674960, 36874965],
       [37874974, 38074984],
       [47475091, 47675093],
       [48675105, 48875113],
       [49775123, 49975126],
       [51075136, 51275136],
       [52175147, 52375146],
       [53175166, 53375161],
       [54175173, 54375171],
       [56275196, 56475198],
       [58675225, 58875229],
       [61075256, 61275258],
       [63275279, 63475282],
       [64975299, 65175302],
       [67375325, 67575330],
       [68575340, 68775345],
       [698753

In [65]:
trodes_state_df["event_frames"] = trodes_state_df.apply(lambda x: utilities.helper.find_nearest_indices(x["event_timestamps"], x["video_timestamps"]), axis=1)

In [66]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps,event_frames
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,[6771359],state,[1],"[1.1, 1.4]",1.1,[],[],20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",[]
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7874619, 8074622], [10274648, 10474653], [12...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[1101, 1300], [3496, 3696], [5792, 5991], [75..."
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7081611, 7085413], [7938621, 7967419], [8333...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[308, 312], [1165, 1194], [1557, 1571], [1643..."
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7081611, 7085413], [7938621, 7967419], [8333...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[308, 312], [1165, 1194], [1557, 1571], [1643..."
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7874619, 8074622], [10274648, 10474653], [12...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[1101, 1300], [3496, 3696], [5792, 5991], [75..."


## Combine raw and state dataframes

In [67]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps,event_frames
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,[6771359],state,[1],"[1.1, 1.4]",1.1,[],[],20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",[]
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7874619, 8074622], [10274648, 10474653], [12...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[1101, 1300], [3496, 3696], [5792, 5991], [75..."
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7081611, 7085413], [7938621, 7967419], [8333...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[308, 312], [1165, 1194], [1557, 1571], [1643..."
3,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7081611, 7085413, 7938621, 7967419, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7081611, 7085413], [7938621, 7967419], [8333...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[308, 312], [1165, 1194], [1557, 1571], [1643..."
4,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[6771359, 7874619, 8074622, 10274648, 10474653...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[1.1, 1.4]",1.4,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[7874619, 8074622], [10274648, 10474653], [12...",20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[1101, 1300], [3496, 3696], [5792, 5991], [75..."
5,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230625_112913_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,[6771359],state,[1],"[1.1, 1.4]",1.4,[],[],20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...",[]
6,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230624_105855_standard_comp_to_both_rewarded...,20000,2.4.0,May 24 2023,...,"[8948560, 10085944, 10091941, 10158345, 102915...",state,"[

In [68]:
trodes_state_df = trodes_state_df[STATE_COLS_TO_KEEP].drop_duplicates(subset=["session_dir", "video_name", "metadata_file"]).sort_values(["session_dir", "video_name", "metadata_file"]).reset_index(drop=True).copy()

In [69]:
trodes_state_df.head()

,session_dir,metadata_file,event_timestamps,video_name,video_timestamps,event_frames
0,20230622_110832_standard_comp_to_both_rewarded...,dio_ECU_Din1,"[[5668537, 5868539], [8068567, 8268570], [1036...",20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[1129, 1328], [3525, 3724], [5820, 6019], [76..."
1,20230622_110832_standard_comp_to_both_rewarded...,dio_ECU_Din2,"[[39285155, 39294156], [42062990, 42066190], [...",20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[35707, 35719], [38646, 38648], [38673, 38683..."
2,20230622_110832_standard_comp_to_both_rewarded...,dio_ECU_Din3,"[[64931479, 64931481], [64931483, 72114981], [...",20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[61471, 61471], [61471, 68640], [68827, 68827..."
3,20230624_105855_standard_comp_to_both_rewarded...,dio_ECU_Din1,"[[10070344, 10270344], [12470372, 12670377], [...",20230624_105855_standard_comp_to_both_rewarded...,"[8949944, 8950349, 8951330, 8952716, 8954102, ...","[[1119, 1318], [3515, 3714], [5810, 6010], [76..."
4,20230624_105855_standard_comp_to_both_rewarded...,dio_ECU_Din2,"[[10085944, 10091941], [10158345, 10291544], [...",20230624_105855_standard_comp_to_both_rewarded...,"[8949944, 8950349, 8951330, 8952716, 8954102, ...","[[1134, 1141], [1206, 1340], [1341, 1342], [13..."


In [70]:
trodes_state_df = trodes_state_df.groupby(same_columns).agg({**{col: 'first' for col in trodes_state_df.columns if col not in same_columns + different_columns}, **{col: lambda x: x.tolist() for col in different_columns}}).reset_index()

In [71]:
trodes_state_df.head()

,session_dir,video_name,video_timestamps,metadata_file,event_frames,event_timestamps
0,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...","[dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]","[[[1129, 1328], [3525, 3724], [5820, 6019], [7...","[[[5668537, 5868539], [8068567, 8268570], [103..."
1,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,"[8949944, 8950349, 8951330, 8952716, 8954102, ...","[dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]","[[[1119, 1318], [3515, 3714], [5810, 6010], [7...","[[[10070344, 10270344], [12470372, 12670377], ..."
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[dio_ECU_Din1, dio_ECU_Din2, dio_ECU_Din3]","[[[1101, 1300], [3496, 3696], [5792, 5991], [7...","[[[7874619, 8074622], [10274648, 10474653], [1..."


In [72]:
trodes_state_df["tone_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[0])
trodes_state_df["box_1_port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[1])
trodes_state_df["box_2_port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[2])

trodes_state_df["tone_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[0])
trodes_state_df["box_1_port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[1])
trodes_state_df["box_2_port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[2])


In [73]:
trodes_state_df = trodes_state_df.drop(columns=["event_timestamps", "event_frames", "metadata_file"], errors="ignore")

In [74]:
trodes_state_df.head()

,session_dir,video_name,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames
0,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,"[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[5668537, 5868539], [8068567, 8268570], [1036...","[[39285155, 39294156], [42062990, 42066190], [...","[[64931479, 64931481], [64931483, 72114981], [...","[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827..."
1,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,"[8949944, 8950349, 8951330, 8952716, 8954102, ...","[[10070344, 10270344], [12470372, 12670377], [...","[[10085944, 10091941], [10158345, 10291544], [...",[],"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[]
2,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,"[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[7874619, 8074622], [10274648, 10474653], [12...","[[7081611, 7085413], [7938621, 7967419], [8333...",[],"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[]


In [75]:
trodes_raw_df = trodes_raw_df[RAW_COLS_TO_KEEP].reset_index(drop=True).copy()

In [76]:
trodes_raw_df.head()

,session_dir,recording,original_file,session_path,current_subject,first_item_data,first_timestamp,all_subjects
0,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,1.1,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",6771359,"[1.1, 1.4]"
1,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,1.4,"[6771359, 6771360, 6771361, 6771362, 6771363, ...",6771359,"[1.1, 1.4]"
2,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,1.4,"[8948560, 8948561, 8948562, 8948563, 8948564, ...",8948560,"[1.2, 1.4]"
3,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,1.1,"[8948560, 8948561, 8948562, 8948563, 8948564, ...",8948560,"[1.2, 1.4]"
4,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,1.2,"[4537337, 4537338, 4537339, 4537340, 4537341, ...",4537337,"[1.1, 1.2]"


In [77]:
trodes_final_df = pd.merge(trodes_raw_df, trodes_state_df, on=["session_dir"], how="inner")

In [78]:
trodes_final_df.shape

(6, 16)

In [79]:
trodes_final_df = trodes_final_df.rename(columns={"first_item_data": "raw_timestamps"})
trodes_final_df = trodes_final_df.drop(columns=["metadata_file"], errors="ignore")
trodes_final_df = trodes_final_df.sort_values(["session_dir", "recording"]).reset_index(drop=True).copy()

## Making the timestamps 0 indexed

In [80]:
trodes_final_df[[col for col in trodes_final_df.columns if "timestamps" in col]].head()

,raw_timestamps,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,"[4537337, 4537338, 4537339, 4537340, 4537341, ...","[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[5668537, 5868539], [8068567, 8268570], [1036...","[[39285155, 39294156], [42062990, 42066190], [...","[[64931479, 64931481], [64931483, 72114981], [..."
1,"[4537337, 4537338, 4537339, 4537340, 4537341, ...","[4537335, 4538721, 4540107, 4541493, 4541493, ...","[[5668537, 5868539], [8068567, 8268570], [1036...","[[39285155, 39294156], [42062990, 42066190], [...","[[64931479, 64931481], [64931483, 72114981], [..."
2,"[8948560, 8948561, 8948562, 8948563, 8948564, ...","[8949944, 8950349, 8951330, 8952716, 8954102, ...","[[10070344, 10270344], [12470372, 12670377], [...","[[10085944, 10091941], [10158345, 10291544], [...",[]
3,"[8948560, 8948561, 8948562, 8948563, 8948564, ...","[8949944, 8950349, 8951330, 8952716, 8954102, ...","[[10070344, 10270344], [12470372, 12670377], [...","[[10085944, 10091941], [10158345, 10291544], [...",[]
4,"[6771359, 6771360, 6771361, 6771362, 6771363, ...","[6772743, 6774129, 6774129, 6775515, 6776901, ...","[[7874619, 8074622], [10274648, 10474653], [12...","[[7081611, 7085413], [7938621, 7967419], [8333...",[]


In [81]:
trodes_final_df["last_timestamp"] = trodes_final_df["raw_timestamps"].apply(lambda x: x[-1])

- Dropping raw timestamps because of memory issues

In [82]:
trodes_final_df = trodes_final_df.drop(columns=["raw_timestamps", "original_file"], errors="ignore")

In [83]:
copy_trodes_final_df = trodes_final_df.copy

In [84]:
for col in [col for col in trodes_final_df.columns if "timestamps" in col]:
    trodes_final_df[col] = trodes_final_df.apply(lambda x: x[col].astype(np.int32) - np.int32(x["first_timestamp"]), axis=1)

for col in [col for col in trodes_final_df.columns if "frames" in col]:
    trodes_final_df[col] = trodes_final_df[col].apply(lambda x: x.astype(np.int32))

In [85]:
sorted_columns = sorted(trodes_final_df.columns
, key=lambda x: x.split("_")[-1])
trodes_final_df = trodes_final_df[sorted_columns].copy()

## Saving to a file

In [86]:
trodes_final_df.to_pickle(os.path.join(OUTPUT_DIR, "{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX)))

In [87]:
trodes_final_df.head()

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.1,"[1.1, 1.2]",4537337,74904369,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [..."
1,20230622_110832_standard_comp_to_both_rewarded...,"[[1129, 1328], [3525, 3724], [5820, 6019], [76...","[[35707, 35719], [38646, 38648], [38673, 38683...","[[61471, 61471], [61471, 68640], [68827, 68827...",20230622_110832_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230622_110832_standard_comp_to_both_rewarded...,1.2,"[1.1, 1.2]",4537337,76591466,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[1131200, 1331202], [3531230, 3731233], [5831...","[[34747818, 34756819], [37525653, 37528853], [...","[[60394142, 60394144], [60394146, 67577644], [..."
2,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.1,"[1.2, 1.4]",8948560,80713042,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[]
3,20230624_105855_standard_comp_to_both_rewarded...,"[[1119, 1318], [3515, 3714], [5810, 6010], [76...","[[1134, 1141], [1206, 1340], [1341, 1342], [13...",[],20230624_105855_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230624_105855_standard_comp_to_both_rewarded...,1.4,"[1.2, 1.4]",8948560,80713042,"[1384, 1789, 2770, 4156, 5542, 5927, 6928, 831...","[[1121784, 1321784], [3521812, 3721817], [5821...","[[1137384, 1143381], [1209785, 1342984], [1344...",[]
4,20230625_112913_standard_comp_to_both_rewarded...,"[[1101, 1300], [3496, 3696], [5792, 5991], [75...","[[308, 312], [1165, 1194], [1557, 1571], [1643...",[],20230625_112913_standard_comp_to_both_rewarded...,/scratch/back_up/reward_competition_extention/...,20230625_112913_standard_comp_to_both_rewarded...,1.1,"[1.1, 1.4]",6771359,79195515,"[1384, 2770, 2770, 4156, 5542, 6928, 6928, 831...","[[1103260, 1303263], [3503289, 3703294], [5803...","[[310252, 314054], [1167262, 1196060], [156166...",[]


In [88]:
trodes_final_df["session_dir"].unique()

array(['20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4',
       '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4'],
      dtype=object)

In [89]:
trodes_final_df["video_name"].unique()

array(['20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4.1.videoTimeStamps.cameraHWSync',
       '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.1.videoTimeStamps.cameraHWSync'],
      dtype=object)

In [90]:
OUTPUT_DIR

'./proc'